In [20]:
import pandas as pd

In [21]:
# display floats with 2 decimal places
pd.options.display.float_format = '{:,.5f}'.format
 
# expand display limits
pd.options.display.max_rows = 200
pd.options.display.max_columns = 100

In [22]:
df = pd.read_csv('all_stocks_2017-2018.csv',
                names=['Date', 'Open', 'High', 'Low', 'Close', 'Volume','Name']).reset_index().drop(0)
df.head()

,index,Date,Open,High,Low,Close,Volume,Name
1,1,1/3/2017,178.83,180,177.22,178.05,2510055,MMM
2,2,1/4/2017,178.03,178.9,177.61,178.32,1541985,MMM
3,3,1/5/2017,178.26,179.14,176.89,177.71,1447848,MMM
4,4,1/6/2017,177.29,178.6,175.8,178.23,1625049,MMM
5,5,1/9/2017,178.37,178.38,177.2,177.27,1622625,MMM


In [23]:
for i in ['Open', 'High', 'Low', 'Close', 'Volume']:
    df.loc[:,i] = df.loc[:,i].astype('float')

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7781 entries, 1 to 7781
Data columns (total 8 columns):
index     7781 non-null int64
Date      7781 non-null object
Open      7757 non-null float64
High      7772 non-null float64
Low       7762 non-null float64
Close     7781 non-null float64
Volume    7781 non-null float64
Name      7781 non-null object
dtypes: float64(5), int64(1), object(2)
memory usage: 547.1+ KB


In [25]:

print( df.Name.unique() )

['MMM' 'AXP' 'AAPL' 'BA' 'CAT' 'CVX' 'CSCO' 'KO' 'DIS' 'XOM' 'GE' 'GS'
 'HD' 'IBM' 'INTC' 'JNJ' 'JPM' 'MCD' 'MRK' 'MSFT' 'NKE' 'PFE' 'PG' 'TRV'
 'UTX' 'UNH' 'VZ' 'WMT' 'GOOGL' 'AMZN' 'AABA']


In [26]:
df_AAPL = df[df.Name.isin(['AAPL','GOOGL','MSFT','AMZN'])]

## One simple pivot do the trick

In [27]:
# pivot the dataset while keeping only one price column
pivoted_df = df_AAPL.pivot(index='Date', columns='Name', values='Close')

# display examples from pivoted dataset
pivoted_df.tail()

Name,AAPL,AMZN,GOOGL,MSFT
Date,,,,
9/29/2017,154.12000,961.35000,973.72000,74.49000
9/5/2017,162.08000,965.27000,941.48000,73.61000
9/6/2017,161.91000,967.80000,942.02000,73.40000
9/7/2017,161.26000,979.47000,949.89000,74.34000
9/8/2017,158.63000,965.90000,941.41000,73.98000


In [28]:
# calculate returns over the prior 7, 14, 21 and 28 days using shift method

# shift the index of the dataframe by some number of periods
pivoted_df.tail(3)

Name,AAPL,AMZN,GOOGL,MSFT
Date,,,,
9/6/2017,161.91000,967.80000,942.02000,73.40000
9/7/2017,161.26000,979.47000,949.89000,74.34000
9/8/2017,158.63000,965.90000,941.41000,73.98000


## Simple and easy in Python with .shift

In [29]:
shifted = pivoted_df.shift(-7)

In [30]:
# to calcuate returns over 7 days prior we can use prices_today/prices_7_days_ago -1.0
#calculate returns over 7 days prior
delta_7 = pivoted_df / shifted - 1.0

#display examples
delta_7.tail()

Name,AAPL,AMZN,GOOGL,MSFT
Date,,,,
9/29/2017,nan,nan,nan,nan
9/5/2017,nan,nan,nan,nan
9/6/2017,nan,nan,nan,nan
9/7/2017,nan,nan,nan,nan
9/8/2017,nan,nan,nan,nan


In [31]:
# calculate returns over each window and store them in dictionary using loop

delta_dict = {}
for offset in [7, 14, 21, 28]:
    delta_dict['delta_{}'.format(offset)] = pivoted_df / pivoted_df.shift(offset) - 1.0

In [32]:
# melt data to create analytical base table (ABT)
# melt delta_7 returns

melted_7 = delta_7.reset_index().melt(id_vars='Date', value_name= 'delta_7')

#melted dataframe examples
melted_7.tail()

,Date,Name,delta_7
999,9/29/2017,MSFT,nan
1000,9/5/2017,MSFT,nan
1001,9/6/2017,MSFT,nan
1002,9/7/2017,MSFT,nan
1003,9/8/2017,MSFT,nan


In [33]:
# to do so for all the returns dataframes, loop through delta_dict

#melt all the delta dataframes and store in list
melted_dfs = []
for key, delta_df in delta_dict.items():
    melted_dfs.append( delta_df.reset_index().melt(id_vars=['Date'], value_name=key))

In [34]:
# calculate 7-day returns after the date
return_df = pivoted_df.shift(-7) / pivoted_df - 1.0

# melt the return dataset and append to list
melted_dfs.append( return_df.reset_index().melt(id_vars=['Date'], value_name='return_7') )

In [35]:
# we now have 5 melted dataframes stred in the melted_dfs list
# now to join melted dataframes into a single ABT 

#merge two dataframes
pd.merge(melted_dfs[0], melted_dfs[1], on=['Date']).tail()

,Date,Name_x,delta_7,Name_y,delta_14
4011,9/8/2017,GOOGL,0.00425,MSFT,-0.01766
4012,9/8/2017,MSFT,0.00983,AAPL,-0.00782
4013,9/8/2017,MSFT,0.00983,AMZN,-0.02117
4014,9/8/2017,MSFT,0.00983,GOOGL,0.00654
4015,9/8/2017,MSFT,0.00983,MSFT,-0.01766


In [36]:
from functools import reduce

In [37]:
# create feature_dfs list that contains base features from original dataset plus melted dataset

# grab features from original dataset
base_df = df[['Date','Name', 'Volume', 'Close']]

# create a list with all the features dataframes
feature_dfs = [base_df] + melted_dfs

In [38]:
# now we can use reduce
# reduce applies to two arguments cummulatively objects and sequence 

# reduce-merge features into analytical bae table
abt = reduce(lambda left,right: pd.merge(left,right,on=['Date','Name']), feature_dfs)

# display examples from the ABT 
abt.tail(10)

,Date,Name,Volume,Close,delta_7,delta_14,delta_21,delta_28,return_7
994,12/15/2017,AMZN,"4,778,621.00000","1,179.14000",0.04083,-0.01396,0.03680,0.19156,0.00265
995,12/18/2017,AMZN,"2,947,625.00000","1,190.58000",0.05442,-0.00253,0.05372,0.20140,-0.00376
996,12/19/2017,AMZN,"2,587,792.00000","1,187.38000",0.02153,0.02248,0.08514,0.07584,-0.01508
997,12/20/2017,AMZN,"2,371,166.00000","1,177.62000",0.00744,0.05939,0.04556,0.04645,-0.03708
998,12/21/2017,AMZN,"2,123,117.00000","1,174.76000",0.00831,-0.00169,0.03095,0.04037,-0.02825
999,12/22/2017,AMZN,"1,585,054.00000","1,168.36000",0.00363,0.04256,0.01055,0.02773,-0.01370
1000,12/26/2017,AMZN,"2,005,187.00000","1,176.76000",0.00213,0.04771,-0.00779,0.04444,-0.01442
1001,12/27/2017,AMZN,"1,867,208.00000","1,182.26000",0.00265,0.04359,-0.01135,0.03954,-0.01714
1002,12/28/2017,AMZN,"1,841,676.00000","1,186.10000",-0.00376,0.05045,-0.00628,0.04976,-0.29825
1003,12/29/2017,AMZN,"2,688,391.00000","1,169.47000",-0.01508,0.00613,0.00706,0.06877,-0.29245
